# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_file = "../Resources/city_weather_data.csv"

# Read our Kickstarter data into pandas
df = pd.read_csv(city_data_file)
df.head()

,Unnamed: 0,City Name,Current Temp,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,0,barrow,19.40,71.2906,-156.7887,19.40,86,90,12.66,US,1619926543
1,1,nome,26.94,64.5011,-165.4064,28.40,69,40,13.80,US,1619926394
2,2,ales,44.80,44.1333,4.0833,46.99,87,0,6.91,FR,1619926544
3,3,georgetown,87.33,5.4112,100.3354,89.60,79,20,8.05,MY,1619926545
4,4,hermanus,57.00,-34.4187,19.2345,57.00,74,85,1.01,ZA,1619926518


In [3]:
city_weather_data_df = df.drop(columns=['Unnamed: 0'])
city_weather_data_df.head()

,City Name,Current Temp,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,barrow,19.40,71.2906,-156.7887,19.40,86,90,12.66,US,1619926543
1,nome,26.94,64.5011,-165.4064,28.40,69,40,13.80,US,1619926394
2,ales,44.80,44.1333,4.0833,46.99,87,0,6.91,FR,1619926544
3,georgetown,87.33,5.4112,100.3354,89.60,79,20,8.05,MY,1619926545
4,hermanus,57.00,-34.4187,19.2345,57.00,74,85,1.01,ZA,1619926518


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure()

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_data_df[["Latitude", "Longitude"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
# census_data_complete = census_data_complete.dropna()
humidity_rate = city_weather_data_df["Humidity"].astype(float)

In [6]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Filter down by 3 conditions for ideal weather
filtered_df = city_weather_data_df.loc[(city_weather_data_df['Max Temp']>=70) & (city_weather_data_df['Max Temp']< 80) & (city_weather_data_df['Wind'] < 10)]

In [8]:
# check to see if there are any NA
filtered_df.count()

City Name       80
Current Temp    80
Latitude        80
Longitude       80
Max Temp        80
Humidity        80
Cloudiness      80
Wind            80
Country         80
Date            80
dtype: int64

In [9]:
# run if there are NaN to eliminate
filtered_df.dropna(how='all')

,City Name,Current Temp,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Country,Date
6,saint-philippe,74.19,-21.3585,55.7679,75.2,64,0,9.22,RE,1619926546
9,hobart,69.46,-42.8794,147.3294,71.6,40,0,6.91,AU,1619926547
15,souillac,73.40,-20.5167,57.5167,73.4,94,40,2.30,MU,1619926549
17,hilo,70.36,19.7297,-155.0900,75.2,83,90,5.75,US,1619926550
18,new norfolk,69.33,-42.7826,147.0587,70.0,39,100,1.01,AU,1619926516
...,...,...,...,...,...,...,...,...,...,...
535,santarem,73.40,-2.4431,-54.7083,73.4,100,75,3.44,BR,1619926879
539,opelousas,70.77,30.5335,-92.0815,72.0,83,40,4.61,US,1619926882
575,victoria point,72.43,-27.5833,153.3000,73.0,71,97,5.01,AU,1619926902
577,shadegan,76.95,30.6497,48.6647,78.8,83,40,9.22,IR,1619926903


In [10]:
filtered_df.count()

City Name       80
Current Temp    80
Latitude        80
Longitude       80
Max Temp        80
Humidity        80
Cloudiness      80
Wind            80
Country         80
Date            80
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# create new hotel data frame and add new column for hotel name
hotel_df = filtered_df

hotel_df['Hotel Name'] = ""

hotel_df.head(10)

<ipython-input-11-c3b73b899dda>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City Name,Current Temp,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel Name
6,saint-philippe,74.19,-21.3585,55.7679,75.20,64,0,9.22,RE,1619926546,
9,hobart,69.46,-42.8794,147.3294,71.60,40,0,6.91,AU,1619926547,
15,souillac,73.40,-20.5167,57.5167,73.40,94,40,2.30,MU,1619926549,
17,hilo,70.36,19.7297,-155.0900,75.20,83,90,5.75,US,1619926550,
18,new norfolk,69.33,-42.7826,147.0587,70.00,39,100,1.01,AU,1619926516,
26,puerto ayora,73.99,-0.7393,-90.3518,73.99,93,36,1.01,EC,1619926554,
42,socorro,75.81,31.6546,-106.3033,78.80,22,20,6.91,US,1619926561,
55,touros,75.20,-5.1989,-35.4608,75.20,94,40,4.61,BR,1619926568,
60,oussouye,70.68,12.4850,-16.5469,71.60,94,0,4.61,SN,1619926570,
61,vila velha,71.69,-20.3297,-40.2925,75.00,100,75,3.44,BR,1619926572,


In [13]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}


print("Begin Hotel Search by Location")
print("------------------------------")

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        print("Hotel Found")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

print("-------------------------------------")
print("Nearest Hotel Search is now complete.")

Begin Hotel Search by Location
------------------------------
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Missing field/result... skipping.
Hotel Found
Missing field/result... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Missing field/result... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Missing field/result... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Missing field/result... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Missing field/result... skipping.
Hotel Found
Missing field/result... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Missing field/result... skipping.
Hotel Found
Missing field/result... skipping.
Missing 

In [14]:
hotel_df.head()

,City Name,Current Temp,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel Name
6,saint-philippe,74.19,-21.3585,55.7679,75.2,64,0,9.22,RE,1619926546,Les Embruns Du Baril
9,hobart,69.46,-42.8794,147.3294,71.6,40,0,6.91,AU,1619926547,MACq 01 Hotel
15,souillac,73.40,-20.5167,57.5167,73.4,94,40,2.30,MU,1619926549,Shanti Maurice Resort & Spa
17,hilo,70.36,19.7297,-155.0900,75.2,83,90,5.75,US,1619926550,Hilo Hawaiian Hotel
18,new norfolk,69.33,-42.7826,147.0587,70.0,39,100,1.01,AU,1619926516,


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
humidity_rate_2 = hotel_df["Humidity"].astype(float)
fig_2 = gmaps.figure()

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heat_layer_2 = gmaps.heatmap_layer(locations, weights=humidity_rate_2, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig_2.add_layer(heat_layer_2)
fig_2.add_layer(marker_layer)
fig_2

Figure(layout=FigureLayout(height='420px'))